In [1]:
using Pkg
Pkg.activate("/home/karen/projects/AutonomousRiskFramework/STLCG.jl/.")

 Activating environment at `~/projects/AutonomousRiskFramework/STLCG.jl/Project.toml`


In [8]:
using Revise
using STLCG
using EllipsisNotation


In [9]:
bs = 2
t = 15
x_dim = 3
dim = 2
x = rand(bs, t, x_dim);
xs = [x[:,i:i,..] for i in 1:t];


op = Always(subformula=x, interval=[5, Inf])
h0 = STLCG.init_rnn_cell!(op, x)
scale = 0
distributed = true

# rnn_cell(op, xs[1], op.h0; scale, distributed)
STLCG.gradient(x -> sum(STLCG.rnn_cell(op, x, h0; scale, distributed)[1]), xs[1]) # <---- doesn't work
# STLCG.gradient(x -> sum(STLCG.rnn_cell(op, x, h0; scale, distributed)[2][1][2]), xs[1]) # <--- works

sdfs


(nothing,)